# Phân Tích Cảm Xúc Comment với Vietnamese-Sentiment-visobert

Notebook này sẽ:
1. Lấy dữ liệu comment từ bảng `nessie.silver_tables.comment` trong khoảng thời gian nhất định
2. Áp dụng model Vietnamese-Sentiment-visobert để phân tích cảm xúc
3. Tổng hợp các chỉ số POS (tích cực), NEG (tiêu cực), NEU (trung lập)

## 1. Import Libraries và Khởi tạo Spark Session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, to_date, avg, sum as spark_sum
from pyspark.sql.types import *
import os
from datetime import datetime
import pandas as pd

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session với Iceberg và Nessie catalog
spark = (
    SparkSession.builder.appName("Apply_Sentiment_Model")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "2g")
    .config("spark.executor.cores", "2")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://silver/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    # ===== Cấu hình MinIO (S3-compatible) =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    # Propagate environment variables to executors
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    # ===== JAR files =====
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("Spark Session đã được khởi tạo!")
print(f"Spark Master: {spark.sparkContext.master}")
print(f"Application ID: {spark.sparkContext.applicationId}")

## 2. Load Vietnamese Sentiment Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Đường dẫn đến model
model_path = "/opt/spark-apps/Vietnamese-Sentiment-visobert"

print("⏳ Đang tải model Vietnamese-Sentiment-visobert...")

# Load tokenizer và model (use_fast=False để tránh lỗi tokenizer.json)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Kiểm tra và sử dụng GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

print(f"Model đã được tải thành công!")
print(f"Device: {device}")
print(f"Labels: {model.config.id2label}")

## 3. Định nghĩa Hàm Dự Đoán Cảm Xúc

In [ ]:
def predict_sentiment(text):
    """
    Dự đoán cảm xúc của văn bản
    Returns: sentiment label (POS, NEG, NEU)
    """
    if not text or text.strip() == "":
        return "NEU"
    
    try:
        # Tokenize
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Dự đoán
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = F.softmax(logits, dim=-1)
            predicted_class = torch.argmax(probabilities, dim=-1).item()
        
        # Chuyển đổi id sang label
        sentiment = model.config.id2label[predicted_class]
        return sentiment
    except Exception as e:
        print(f"Error processing text: {str(e)}")
        return "NEU"

# Test function
test_text = "Trường này rất tốt, giảng viên nhiệt tình!"
print(f"Test text: {test_text}")
print(f"Sentiment: {predict_sentiment(test_text)}")

## 4. Lấy Dữ Liệu Comment từ Bảng Silver (Batch Processing)

In [ ]:
# Cấu hình batch processing
BATCH_SIZE = 20000  # Số lượng comment xử lý mỗi batch

print("🔍 Đang load batch comment chưa xử lý từ Silver...")

# ===== TỐI ƯU: Dùng SQL với LEFT ANTI JOIN (SUBQUERY) =====
# Cách này NHANH NHẤT vì:
# 1. Spark optimize toàn bộ query thành 1 execution plan
# 2. Iceberg pruning: chỉ scan partition/file cần thiết
# 3. KHÔNG load toàn bộ Silver hay Gold vào memory
# 4. Push-down predicates xuống storage layer

try:
    # Query với LEFT ANTI JOIN ngay trong SQL
    # Spark sẽ optimize và chỉ scan cần thiết
    df_comments = spark.sql(f"""
        SELECT 
            s.articleID as postID,
            s.commentID,
            s.comment,
            s.commentTime,
            s.commentLike,
            s.levelComment,
            s.numberOfReply,
            s.created_at,
            s.updated_at
        FROM nessie.silver_tables.comment s
        LEFT ANTI JOIN nessie.gold_result_model_multi_task.Comment_Sentiment g
            ON s.commentID = g.commentID
        WHERE s.comment IS NOT NULL
          AND TRIM(s.comment) != ''
        LIMIT {BATCH_SIZE}
    """)
    
    total_comments = df_comments.count()
    
    # # Đếm số comment đã xử lý (nếu cần thống kê)
    # try:
    #     existing_count = spark.sql("""
    #         SELECT COUNT(DISTINCT commentID) as count
    #         FROM nessie.gold_result_model_multi_task.Comment_Sentiment
    #     """).collect()[0]['count']
    #     print(f"Đã có trong Gold: {existing_count:,} comment")
    # except:
    #     print("Bảng Gold đang trống")
    
    print(f"📦 Batch hiện tại: {total_comments:,} comment")
    
except Exception as e:
    print(f"⚠️ Bảng Gold chưa tồn tại: {str(e)}")
    print("📝 Load batch đầu tiên từ Silver...")
    
    # Nếu Gold chưa có, lấy batch đầu tiên
    df_comments = spark.sql(f"""
        SELECT 
            articleID as postID,
            commentID,
            comment,
            commentTime,
            commentLike,
            levelComment,
            numberOfReply,
            created_at,
            updated_at
        FROM nessie.silver_tables.comment
        WHERE comment IS NOT NULL
          AND TRIM(comment) != ''
        LIMIT {BATCH_SIZE}
    """)
    
    total_comments = df_comments.count()
    print(f"📦 Batch đầu tiên: {total_comments:,} comment")

if total_comments > 0:
    print(f"\nSẵn sàng xử lý {total_comments:,} comment")
else:
    print("\n🎉 Tất cả comment đã được xử lý!")
    print("Không có comment mới cần phân tích.")

## 5. Áp Dụng Model Sentiment cho Comments

In [ ]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, TimestampType

import pandas as pd
from datetime import datetime

# ===== HÀM XỬ LÝ GIÁ TRỊ TIMESTAMP =====
def safe_ts(v):
    """Chuyển pandas NaT / NaN / None thành None hoặc datetime."""
    if v is None or pd.isna(v):
        return None
    if isinstance(v, pd.Timestamp):
        return v.to_pydatetime()
    return v

print("Đang áp dụng model sentiment...")

if total_comments > 0:
    print("\n📥 Bước 1: Thu thập dữ liệu về driver...")
    comments_pd = df_comments.toPandas()
    print(f"Thu thập {len(comments_pd)} comment")

    batch_save_size = 1000
    results = []

    schema = StructType([
        StructField("postID", StringType(), True),
        StructField("commentID", LongType(), True),
        StructField("comment", StringType(), True),
        StructField("sentiment", StringType(), True),
        StructField("commentTime", TimestampType(), True),
        StructField("commentLike", IntegerType(), True),
        StructField("levelComment", IntegerType(), True),
        StructField("numberOfReply", IntegerType(), True),
        StructField("created_at", TimestampType(), True),
        StructField("updated_at", TimestampType(), True)
    ])

    print("\nBước 2: Chuẩn bị bảng Gold...")
    spark.sql("CREATE DATABASE IF NOT EXISTS nessie.gold_result_model_multi_task")

    spark.sql("""
    CREATE TABLE IF NOT EXISTS nessie.gold_result_model_multi_task.Comment_Sentiment (
        postID STRING,
        commentID BIGINT,
        comment STRING,
        sentiment STRING,
        commentTime TIMESTAMP,
        commentLike INT,
        levelComment INT,
        numberOfReply INT,
        created_at TIMESTAMP,
        updated_at TIMESTAMP
    ) USING iceberg
    """)

    print("Bảng đã tồn tại hoặc được tạo mới.")

    print(f"\n Bước 3: Xử lý {len(comments_pd)} comment...")

    for idx, row in comments_pd.iterrows():

        # ---- Predict sentiment ----
        sentiment = predict_sentiment(row['comment'])

        # ---- Append kết quả vào list ----
        results.append({
            'postID': row['postID'],
            'commentID': int(row['commentID']) if not pd.isna(row['commentID']) else None,
            'comment': row['comment'],
            'sentiment': sentiment,
            'commentTime': safe_ts(row['commentTime']),
            'commentLike': int(row['commentLike']) if not pd.isna(row['commentLike']) else 0,
            'levelComment': int(row['levelComment']) if not pd.isna(row['levelComment']) else 0,
            'numberOfReply': int(row['numberOfReply']) if not pd.isna(row['numberOfReply']) else 0,
            'created_at': safe_ts(row['created_at']),
            'updated_at': datetime.now()
        })

        if (idx + 1) % 500 == 0:
            print(f"   ✓ Đã xử lý {idx+1}/{len(comments_pd)} comment...")

        # ---- Lưu batch ----
        if len(results) >= batch_save_size:
            print(f"\nLưu batch {len(results)} vào Gold...")
            df_batch = spark.createDataFrame(results, schema=schema)
            df_batch.writeTo("nessie.gold_result_model_multi_task.Comment_Sentiment") \
                .using("iceberg") \
                .append()
            print("   → Đã lưu xong batch")
            results = []

    # ---- Lưu batch cuối ----
    if len(results) > 0:
        print(f"\nLưu batch cuối {len(results)} comment...")
        df_batch = spark.createDataFrame(results, schema=schema)
        df_batch.writeTo("nessie.gold_result_model_multi_task.Comment_Sentiment") \
            .using("iceberg") \
            .append()
        print("   → Đã lưu xong batch cuối")

    print("\nHoàn tất phân tích sentiment và lưu Gold layer!")

    print("\nVí dụ 10 dòng mới nhất từ Gold:")
    df_gold_sample = spark.sql("""
        SELECT postID, commentID, comment, sentiment, commentTime, commentLike
        FROM nessie.gold_result_model_multi_task.Comment_Sentiment
        ORDER BY updated_at DESC
        LIMIT 10
    """)
    df_gold_sample.show(10, truncate=50)

    df_with_sentiment = spark.sql("""
        SELECT * FROM nessie.gold_result_model_multi_task.Comment_Sentiment
        ORDER BY updated_at DESC
        LIMIT {}
    """.format(total_comments))

else:
    print("Không có comment để xử lý!")


## 6. Dừng Spark Session

In [ ]:
# Giải phóng cache
if total_comments > 0:
    df_with_sentiment.unpersist()

# Dừng Spark Session
spark.stop()
print("🛑 Spark Session đã được dừng!")
print("✅ Hoàn tất!")